# 資料清洗
科系名稱裡面會出現排版用的空白字元 '\3000'

我也發現有些科系名稱裡面會有 0, [101年度] 此類字樣

有些學校會改名，比如2017年之後`新竹教育大學`的校名欄位變成`清華大學南大校區(原：新竹教育大學)`


# 資料來源
基本上只採集學士資料，為什麼?
因為在台灣碩士跟博士還有在職專班比較容易出現非應屆畢誒的問題
大部分的學士都是直接應屆高中上來(假設忽略重考)

In [1]:
import re
def countDuplicate(arr: [], removeSpace=True ):
    hs = dict()
    for val in arr:
        if isinstance(val,str) and removeSpace:
            val = "".join(val.split())
        hs[val] = hs.get(val,0) + 1
    return hs

def formatDataframeString(df, headNum=0):
    ''' clear all whitespaces in string fileds'''
    rows = ds.shape[0] if headNum<=0 else headNum
    cols = ds.shape[1]
    strColumns = [ i for i in ds.columns if isinstance(i,str)]
    for r in range(rows):
        for c in ds.columns:
            if isinstance(ds.at[r,c],str):
                ds.at[r,c] = "".join(ds.at[r,c].split())
    return df

# 資料匯入

In [2]:
import pandas as pd
fileName = './data/target.xlsx'

# only include the interested columns
ds = pd.read_excel('./data/target.xlsx')[['年度','學校名稱','科系代碼','科系名稱','等級別','總計','男生計','女生計']]
formatDataframeString(ds)


print("Dimension of {}: {}".format(fileName, ds.shape ) )
print("每列資料情況")
ds.head(15)

Dimension of ./data/target.xlsx: (109785, 8)
每列資料情況


,年度,學校名稱,科系代碼,科系名稱,等級別,總計,男生計,女生計
0,2018,政治大學,01111001,教育學系,博士,111,52,59
1,2018,政治大學,01111001,教育學系,碩士,45,14,31
2,2018,政治大學,01111001,教育學系,學士,239,71,168
3,2018,政治大學,01114005,教育行政與政策研究所,碩士,46,16,30
4,2018,政治大學,01114006,學校行政碩士在職專班,碩士,81,26,55
5,2018,政治大學,01121004,幼兒教育研究所,碩士,30,5,25
6,2018,政治大學,02211019,宗教研究所,博士,32,19,13
7,2018,政治大學,02211019,宗教研究所,碩士,28,12,16
8,2018,政治大學,02221020,歷史學系,博士,31,19,12
9,2018,政治大學,02221020,歷史學系,碩士,69,52,17


# 初步清洗資料(去除學位等級別)
要注意同一間學校下面相同科系代碼會對應到不同科系名稱
只取學士資料作分析。

轉換欄位為英文，比較好寫程式。

In [22]:
translated_name = {
    '年度':'year',
    '學校名稱': 'uniName',
    '科系代碼': 'depId',
    '科系名稱': 'depName',
    '等級別': 'degree',
    '總計': 'total',
    '男生計': 'male',
    '女生計': 'female'
}
ds.rename(columns = translated_name, inplace=True)

def filterExactCondition(df, dic):
    filteredTable = df
    for k,v in dic.items():
        filteredTable = filteredTable[ filteredTable[k] == v ]
    return filteredTable

bachelors = filterExactCondition(ds,{'degree':'學士'})
bachelors.tail(15)


,year,uniName,depId,depName,degree,total,male,female
99373,2009,台北市立教育大學,210401,音樂學系,學士,186,17,169
99376,2009,台北市立教育大學,210604,視覺藝術學系,學士,156,33,123
99381,2009,台北市立教育大學,220201,中國語文學系,學士,226,60,166
99384,2009,台北市立教育大學,220906,歷史與地理學系,學士,174,90,84
99386,2009,台北市立教育大學,310509,心理與諮商學系,學士,155,28,127
99388,2009,台北市立教育大學,310809,社會暨公共事務學系,學士,168,48,120
99390,2009,台北市立教育大學,520116,資訊科學系,學士,109,82,27
99398,2009,台北市立體育學院,210511,舞蹈系,學士,158,13,145
99399,2009,台北市立體育學院,210714,動態藝術學系,學士,120,64,56
99402,2009,台北市立體育學院,810305,球類運動學系,學士,501,359,142


# 分群安排

In [106]:
from pprint import pprint
bachByYear = { i: filterExactCondition(bachelors,{'year':i}) for i in bachelors['year'].unique().tolist() }
bachByYear[2017]

uniStaticsByYear = {}


def getByYear():
    unis = {}
    for year in [2017]:
        for uniName in bachelors['uniName'].unique().tolist():
            male, female,total = 0,0,0
            uni = {}
            for dep in filterExactCondition( bachByYear[year], {'uniName':uniName}).itertuples():
                male += getattr(dep, 'male')
                female += getattr(dep,'female')
                total += getattr(dep,'total')
            uni['total'] = total
            uni['male'] = male
            uni['femle'] = female
            uni['uni'] = uniName
            uni['year'] =  year
            unis[uniName] = uni
    return unis

for uni, data in getByYear().items():
    if( data['total'] < 100):
        print(data)
#filterExactCondition( ds,{'uniName':'台灣體育學院'})

{'total': 63, 'male': 23, 'femle': 40, 'uni': '法鼓文理學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '清華大學南大校區(原：新竹教育大學)', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '新竹教育大學', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '中信金融管理學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '興國管理學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '屏東教育大學', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '台北市立教育大學', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '台北市立體育學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '台灣體育學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '立德大學', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '致遠管理學院', 'year': 2017}
{'total': 0, 'male': 0, 'femle': 0, 'uni': '法鼓佛教學院', 'year': 2017}


In [114]:
da = filterExactCondition( ds,{'uniName':'新竹教育大學'})
#getattr( da.iloc[1], 'degree')

,year,uniName,depId,depName,degree,total,male,female
33125,2015,新竹教育大學,140108,教育心理與諮商學系,碩士,77,18,59
33126,2015,新竹教育大學,140108,教育心理與諮商學系,學士,175,36,139
33127,2015,新竹教育大學,140108,教育心理與諮商學系,碩士,87,11,76
33128,2015,新竹教育大學,140122,教育與學習科技學系,博士,58,26,32
33129,2015,新竹教育大學,140122,教育與學習科技學系,碩士,106,32,74
33130,2015,新竹教育大學,140122,教育與學習科技學系,學士,351,83,268
33131,2015,新竹教育大學,140122,教育與學習科技學系,碩士,141,32,109
33132,2015,新竹教育大學,140209,數理教育研究所,碩士,60,31,29
33133,2015,新竹教育大學,140209,數理教育研究所,碩士,73,16,57
33134,2015,新竹教育大學,140212,英語教學系,碩士,4,0,4


# [研究] 科系代碼的意義
`科系代碼`數量 `科系名稱` 數量不同
看起來是一對多關係。
### 結論
2017年之後的科系代碼跟以前便制完全不一樣。
2017年之前的科系代碼跟以前完全相同

In [7]:
from pprint import pprint
print( "科系代碼數量: {}".format(len(countDuplicate(ds['depId']).keys())) )
print( "科系數量: {}".format(len(countDuplicate(ds['depName']).keys())))


def collapse( ds, toIdx, fromIdx, headNum=0 ):
    # 收集在 toIdx 欄位尚有相同值的row, 同時紀錄對應 fromIdx 欄位中的不重複資料
    if headNum <= 0:
        headNum = ds.shape[0] #total number of rows in ds
    todict = dict( [] )
    for index, row in ds.head(headNum).iterrows():
        arr = todict.get(row[toIdx], [])
        if row[fromIdx] not in arr:
            arr.append(row[fromIdx])
        todict[row[toIdx]] = arr
    return todict

#dID_dname = collapse(ds, 'depId', 'depName',1000)
#uni_dID = collapse(ds, 'uniName','depId', 1000)
#dID_uni = collapse(ds, 'depId','uniName',2000)
dname_uni = collapse(ds, 'depName','uniName')
a = dname_uni
for k,v in a.items():
    # 秀出來哪些學校有 "資訊工程學系" (完全符合)
    if len(v)>2 and k=="資訊工程學系":
        print("{}: {}".format(k,v))

科系代碼數量: 5001
科系數量: 10618
資訊工程學系: ['清華大學', '台灣大學', '台灣師範大學', '成功大學', '交通大學', '中央大學', '中山大學', '台灣海洋大學', '中正大學', '彰化師範大學', '台北大學', '嘉義大學', '高雄大學', '東華大學', '暨南國際大學', '台東大學', '宜蘭大學', '聯合大學', '台南大學', '台中教育大學', '金門大學', '屏東大學', '東海大學', '輔仁大學', '中原大學', '淡江大學', '中國文化大學', '逢甲大學', '靜宜大學', '長庚大學', '元智大學', '中華大學', '大葉大學', '義守大學', '銘傳大學', '南華大學', '真理大學', '大同大學', '長榮大學', '亞洲大學', '明新科技大學', '南榮科技大學', '健行科技大學', '正修科技大學', '元培科技大學', '景文科技大學', '虎尾科技大學', '清雲科技大學', '開南大學', '明道大學', '立德大學']


In [20]:
## 查看特定學校 資工系學士 歷年科系代碼變化
def filterExactCondition(df, dic):
    filteredTable = df
    for k,v in dic.items():
        filteredTable = filteredTable[ filteredTable[k] == v ]
    return filteredTable
filterDataFrame(ds, {'depName':'資訊工程學系', 'uniName':'中華大學', 'degree':'學士'})

,year,uniName,depId,depName,degree,total,male,female
5424,2018,中華大學,06131025,資訊工程學系,學士,391,343,48
15209,2017,中華大學,07141025,資訊工程學系,學士,386,334,52
34771,2015,中華大學,520114,資訊工程學系,學士,401,356,45
44446,2014,中華大學,520114,資訊工程學系,學士,393,341,52
54307,2013,中華大學,520114,資訊工程學系,學士,397,344,53
64293,2012,中華大學,520114,資訊工程學系,學士,434,372,62
74233,2011,中華大學,520114,資訊工程學系,學士,506,431,75
84273,2010,中華大學,520114,資訊工程學系,學士,614,526,88
94395,2009,中華大學,520114,資訊工程學系,學士,659,568,91
